# PDU - praca domowa 4
## Michał Wdowski

Tematem czwartego projektu z PDU było odtworzenie siedmiu zapytań `SQL` w języku `Python` i pakietów `pandas` oraz `numpy`. Dane będą pochodzić z dostarczonych uproszczonych zasobów z serwisu *StackExchange*.

In [1]:
# import bibiliotek
import pandas as pd
import numpy as np

# wyłączenie "skracania" długich napisów przy wyświetlaniu ramek danych 
pd.set_option('max_colwidth', -1)

# import danych
Votes = pd.read_csv("Votes.csv.gz", comment="#", compression="gzip")
Posts = pd.read_csv("Posts.csv.gz", comment="#", compression="gzip")
Comments = pd.read_csv("Comments.csv.gz", comment="#", compression="gzip")
Users = pd.read_csv("Users.csv.gz", comment="#", compression="gzip")
Badges = pd.read_csv("Badges.csv.gz", comment="#", compression="gzip")
PostLinks = pd.read_csv("PostLinks.csv.gz", comment="#", compression="gzip")
Tags = pd.read_csv("Tags.csv.gz", comment="#", compression="gzip")

Na potrzeby porównania wyników zostanie utworzona baza danych, do której będą się odnosić zapytania `SQL`.

In [2]:
import tempfile, sqlite3, os, os.path

tempfile.mkdtemp()

baza = os.path.join(tempfile.mkdtemp(), "nycflights13.db")
if os.path.isfile(baza):
    os.remove(baza)

conn = sqlite3.connect(baza)

Votes.to_sql("Votes", conn)
Posts.to_sql("Posts", conn)
Comments.to_sql("Comments", conn)
Users.to_sql("Users", conn)
Badges.to_sql("Badges", conn)
PostLinks.to_sql("PostLinks", conn)
Tags.to_sql("Tags", conn)

## Funkcja 1
Docelowe zapytanie i wynikowa tabela wyglądają następująco:

In [3]:
Cel = pd.read_sql_query("""
                        SELECT
                            Users.DisplayName,
                            Users.Age,
                            Users.Location,
                            SUM(Posts.FavoriteCount) AS FavoriteTotal,
                            Posts.Title AS MostFavoriteQuestion,
                            MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
                        FROM Posts JOIN Users ON Users.Id=Posts.OwnerUserId
                        WHERE Posts.PostTypeId=1
                        GROUP BY OwnerUserId
                        ORDER BY FavoriteTotal DESC
                        LIMIT 10
                        """, conn)
Cel

,DisplayName,Age,Location,FavoriteTotal,MostFavoriteQuestion,MostFavoriteQuestionLikes
0,Mark Mayo,37.0,"Sydney, New South Wales, Australia",467.0,Tactics to avoid getting harassed by corrupt police?,42.0
1,hippietrail,NaN,"Oaxaca, Mexico",444.0,"OK we're all adults here, so really, how on earth should I use a squat toilet?",79.0
2,RoflcoptrException,NaN,None,294.0,How to avoid drinking vodka?,29.0
3,JonathanReez,26.0,"Prague, Czech Republic",221.0,What is the highest viewing spot in London that is accessible free of charge?,17.0
4,nsn,NaN,None,214.0,How do airlines determine ticket prices?,40.0
5,Gagravarr,NaN,"Oxford, United Kingdom",151.0,Are there other places with gardens like those at Versailles?,10.0
6,Andrew Grimm,38.0,"Sydney, Australia",120.0,"OK we're all nerds here, so really, how on earth should I use a Japanese toilet?",8.0
7,VMAtm,33.0,"Tampa, FL, United States",109.0,Is there a good website to plan a trip via trains in Europe?,34.0
8,jrdioko,NaN,None,100.0,What is the most comfortable way to sleep on a plane?,21.0
9,Gayot Fow,NaN,"London, United Kingdom",98.0,Should I submit bank statements when applying for a UK Visa? What do they say about me?,18.0


To zapytanie pokazuje dane *użytkownikow*, których *posty* typu *"Question"* zdobyły najwięcej *polubień*. Oprócz tego pokazuje ich najbardziej lubiane pytanie.

Rozwiązanie w `pandas` może wyglądać następująco:

In [4]:
def function1(Users, Posts):
    
    # Połączenie tabel Posts (dla postów typu 1) i Users
    Posts = Posts[ Posts.PostTypeId==1 ][["Id", "Title", "OwnerUserId", "FavoriteCount"]]
    Posts = Posts.merge(Users, left_on="OwnerUserId", right_on="Id", how="inner")
    
    # Stworzenie zestawienia sumy Favorites dla każdego użytownika
    UsersSum = Posts.groupby("Id_y")["FavoriteCount"].sum().reset_index()
    UsersSum.columns = ["UserId", "FavoriteTotal"]
    
    # Stworzenie zestawienia postów z największym FavoriteCount dla użytkowników
    UsersMax = Posts[["Id_y", "DisplayName", "Age", "Location", "Title", "FavoriteCount"]]
    
    # Grupowanie po Id_y i wybranie wiersza z maksymalnym FavoriteCount - zapisujemy indeksy, które to spełniają
    idx = UsersMax.groupby("Id_y")["FavoriteCount"].transform(max) == UsersMax["FavoriteCount"]
    
    # Wybranie wierszy i kolumn, reset indeksów
    UsersMax = UsersMax[idx].reset_index()#[["Title", "Year", "Count"]]
    
    # Połączenie UsersfSum i UsersMax, posortowanie i wybranie top 10 wyników
    UsersSum = UsersSum.merge(UsersMax, left_on="UserId", right_on="Id_y", how="inner")
    UsersSum = UsersSum.sort_values("FavoriteTotal", ascending=False)
    UsersSum = UsersSum.head(10).reset_index()
    UsersSum = UsersSum[["DisplayName", "Age", "Location", "FavoriteTotal", "Title", "FavoriteCount"]]
    UsersSum.columns = ["DisplayName", "Age", "Location", "FavoriteTotal",
                        "MostFavoriteQuestion", "MostFavoriteQuestionLikes"]
    
    return UsersSum

function1(Users, Posts)

,DisplayName,Age,Location,FavoriteTotal,MostFavoriteQuestion,MostFavoriteQuestionLikes
0,Mark Mayo,37.0,"Sydney, New South Wales, Australia",467.0,Tactics to avoid getting harassed by corrupt police?,42.0
1,hippietrail,NaN,"Oaxaca, Mexico",444.0,"OK we're all adults here, so really, how on earth should I use a squat toilet?",79.0
2,RoflcoptrException,NaN,NaN,294.0,How to avoid drinking vodka?,29.0
3,JonathanReez,26.0,"Prague, Czech Republic",221.0,What is the highest viewing spot in London that is accessible free of charge?,17.0
4,nsn,NaN,NaN,214.0,How do airlines determine ticket prices?,40.0
5,Gagravarr,NaN,"Oxford, United Kingdom",151.0,Are there other places with gardens like those at Versailles?,10.0
6,Andrew Grimm,38.0,"Sydney, Australia",120.0,"OK we're all nerds here, so really, how on earth should I use a Japanese toilet?",8.0
7,VMAtm,33.0,"Tampa, FL, United States",109.0,Is there a good website to plan a trip via trains in Europe?,34.0
8,jrdioko,NaN,NaN,100.0,What is the most comfortable way to sleep on a plane?,21.0
9,Gayot Fow,NaN,"London, United Kingdom",98.0,Should I submit bank statements when applying for a UK Visa? What do they say about me?,18.0


Sprawdźmy czy te ramki danych są identyczne:

In [5]:
function1(Users, Posts).equals(Cel)

True

## Funkcja 2
Docelowe zapytanie i wynikowa tabela wyglądają następująco:

In [6]:
Cel = pd.read_sql_query("""
                        SELECT
                            Posts.ID,
                            Posts.Title,
                            Posts2.PositiveAnswerCount
                        FROM Posts
                        JOIN (
                            SELECT
                                Posts.ParentID,
                                COUNT(*) AS PositiveAnswerCount
                            FROM Posts
                            WHERE Posts.PostTypeID=2 AND Posts.Score>0
                            GROUP BY Posts.ParentID
                        ) AS Posts2
                        ON Posts.ID=Posts2.ParentID
                        ORDER BY Posts2.PositiveAnswerCount DESC
                        LIMIT 10
                        """, conn)
Cel

,Id,Title,PositiveAnswerCount
0,250,Which European cities have bike rental stations for tourists?,24
1,10,"When traveling to a country with a different currency, how should you take your money?",20
2,13562,How do you choose a restaurant when travelling?,20
3,48775,How can I deal with people asking to switch seats with me on a plane?,20
4,3220,Why would you wrap your luggage in plastic?,19
5,43660,Traveling in Europe Solo - 18 years old. Feasible?,19
6,30656,Long-life SIM cards in Europe,18
7,7663,Am I expected to tip wait staff in Europe?,17
8,59128,"Is there a way to prevent ""looking like a tourist"" in order to not be harassed?",17
9,60446,Is it rude to ask if the food contains pork or alcohol?,17


To zapytanie pokazuje *ID* i *tytuł* *postów*, które mają najwięcej pozytywnie ocenianych odpowiedzi.

Rozwiązanie w `pandas` może wyglądać następująco:

In [7]:
def function2(Posts):
    
    # Wybranie postów typu 2, które mają pozytywny Score
    Posts2 = Posts[ (Posts.PostTypeId==2) & (Posts.Score>0) ]
    
    # Utworzenie zestawienia liczby postów dla każdego ParentId i zmiana nazw kolumn
    Posts2 = Posts2.groupby("ParentId").size().reset_index()
    Posts2.columns=["ParentId", "PositiveAnswerCount"]
    
    # Połączenie z tabelą Posts
    Posts = Posts.merge(Posts2, left_on="Id", right_on="ParentId", how="inner")
    
    # Wybranie kolumn i posortowanie; wybór pierwszych 10 rekordów
    Posts = Posts.sort_values("PositiveAnswerCount", ascending=False)
    Posts = Posts.head(10).reset_index()[["Id", "Title", "PositiveAnswerCount"]]
    
    return Posts

function2(Posts)

,Id,Title,PositiveAnswerCount
0,250,Which European cities have bike rental stations for tourists?,24
1,10,"When traveling to a country with a different currency, how should you take your money?",20
2,13562,How do you choose a restaurant when travelling?,20
3,48775,How can I deal with people asking to switch seats with me on a plane?,20
4,43660,Traveling in Europe Solo - 18 years old. Feasible?,19
5,3220,Why would you wrap your luggage in plastic?,19
6,30656,Long-life SIM cards in Europe,18
7,80329,How to cope with too slow Wi-Fi at hotel?,17
8,7663,Am I expected to tip wait staff in Europe?,17
9,59128,"Is there a way to prevent ""looking like a tourist"" in order to not be harassed?",17


Sprawdźmy czy te ramki danych są identyczne:

In [8]:
function2(Posts).equals(Cel)

False

Odpowiedź wyszła fałszywa, co widać również gołym okiem przy porównaniu tych ramek - np. w wierszu o indeksie 4. Bierze się to stąd, że w obrębie jednego wyniku *PositiveAnswerCount* pojawiają się różne kolejności pytań. W sortowaniu ważna była tylko wartość *PositiveAnswerCount*, dlatego pojawiły się różnice. Gdy zignorujemy ten szczegół, te tabele są identyczne.

Alternatywnie, można to naprawić, wprowadzając pewnie dodatkowe sortowanie na niższym poziomie niż po *PositiveAnswerCount* (lub czymkolwiek innym - w późniejszych przykładach będzie pojawiać się ten sam błąd, który również będzie się dał naprawić wprowadzeniem dodatkowego sortowania):

In [9]:
# Dodane zostało w linijce z "ORDER BY" "...Posts.ID DESC"
Cel = pd.read_sql_query("""
                        SELECT
                            Posts.ID,
                            Posts.Title,
                            Posts2.PositiveAnswerCount
                        FROM Posts
                        JOIN (
                            SELECT
                                Posts.ParentID,
                                COUNT(*) AS PositiveAnswerCount
                            FROM Posts
                            WHERE Posts.PostTypeID=2 AND Posts.Score>0
                            GROUP BY Posts.ParentID
                        ) AS Posts2
                        ON Posts.ID=Posts2.ParentID
                        ORDER BY Posts2.PositiveAnswerCount DESC, Posts.ID DESC
                        LIMIT 10
                        """, conn)
Cel

,Id,Title,PositiveAnswerCount
0,250,Which European cities have bike rental stations for tourists?,24
1,48775,How can I deal with people asking to switch seats with me on a plane?,20
2,13562,How do you choose a restaurant when travelling?,20
3,10,"When traveling to a country with a different currency, how should you take your money?",20
4,43660,Traveling in Europe Solo - 18 years old. Feasible?,19
5,3220,Why would you wrap your luggage in plastic?,19
6,30656,Long-life SIM cards in Europe,18
7,80329,How to cope with too slow Wi-Fi at hotel?,17
8,60446,Is it rude to ask if the food contains pork or alcohol?,17
9,59128,"Is there a way to prevent ""looking like a tourist"" in order to not be harassed?",17


In [10]:
def function2_remade(Posts):
    
    # Wybranie postów typu 2, które mają pozytywny Score
    Posts2 = Posts[ (Posts.PostTypeId==2) & (Posts.Score>0) ]
    
    # Utworzenie zestawienia liczby postów dla każdego ParentId i zmiana nazw kolumn
    Posts2 = Posts2.groupby("ParentId").size().reset_index()
    Posts2.columns=["ParentId", "PositiveAnswerCount"]
    
    # Połączenie z tabelą Posts
    Posts = Posts.merge(Posts2, left_on="Id", right_on="ParentId", how="inner")
    
    # Wybranie kolumn i posortowanie; wybór pierwszych 10 rekordów
    # Dodane zostało "Id" w metodzie "sort_values"
    Posts = Posts.sort_values(["PositiveAnswerCount", "Id"], ascending=False)
    Posts = Posts.head(10).reset_index()[["Id", "Title", "PositiveAnswerCount"]]
    
    return Posts

function2_remade(Posts)

,Id,Title,PositiveAnswerCount
0,250,Which European cities have bike rental stations for tourists?,24
1,48775,How can I deal with people asking to switch seats with me on a plane?,20
2,13562,How do you choose a restaurant when travelling?,20
3,10,"When traveling to a country with a different currency, how should you take your money?",20
4,43660,Traveling in Europe Solo - 18 years old. Feasible?,19
5,3220,Why would you wrap your luggage in plastic?,19
6,30656,Long-life SIM cards in Europe,18
7,80329,How to cope with too slow Wi-Fi at hotel?,17
8,60446,Is it rude to ask if the food contains pork or alcohol?,17
9,59128,"Is there a way to prevent ""looking like a tourist"" in order to not be harassed?",17


In [11]:
function2_remade(Posts).equals(Cel)

True

Jak widać, po dodaniu dodatkowego sortowania (w tym wypadku po *Id*), kolejność w obrębie jednego wyniku *PositiveAnswerCount* została ujednolicona. Dzięki temu otrzymane tabele są identyczne.

W ten sam sposób można poradzić sobie z tym błędem w każdym zadaniu, jednak dla zachowania przejrzystości niech tylko zadanie 2 pozostanie naprawione do końca, jako przykład, że uzyskanie wartości `True` dla wywołania metody `equals` jest możliwe, przy odpowiednim przerobieniu wymaganych zapytań.

## Funkcja 3
Docelowe zapytanie i wynikowa tabela wyglądają następująco:

In [12]:
Cel = pd.read_sql_query("""
                        SELECT
                            Posts.Title,
                            UpVotesPerYear.Year,
                            MAX(UpVotesPerYear.Count) AS Count
                        FROM (
                            SELECT
                                PostId,
                                COUNT(*) AS Count,
                                STRFTIME('%Y', Votes.CreationDate) AS Year
                            FROM Votes
                            WHERE VoteTypeId=2
                            GROUP BY PostId, Year
                        ) AS UpVotesPerYear
                        JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
                        WHERE Posts.PostTypeId=1
                        GROUP BY Year
                        """, conn)
Cel

,Title,Year,Count
0,"OK we're all adults here, so really, how on earth should I use a squat toilet?",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up window shades during takeoff and landing?,2013,103
3,How do you know if Americans genuinely/literally mean what they say?,2014,180
4,Immigration officer that stopped me at the airport is texting me. What do I do?,2015,119
5,I don't know my nationality. How can I visit Denmark?,2016,135
6,Why prohibit engine braking?,2017,178


To zapytanie dla każdego roku pokazuje *pytanie*, ktore miało najwięcej *głosów* typu *2* (wraz z wynikiem).

Rozwiązanie w `pandas` może wyglądać następująco:

In [13]:
def function3(Posts, Votes):
    
    # Wybranie postów typu 2
    UpVotesPerYear = Votes[ Votes.VoteTypeId==2 ]
    
    # Utworzenie kolumny Year - "assign" jest czymś jak "mutate" w dplyr
    UpVotesPerYear = UpVotesPerYear.assign( Year=UpVotesPerYear.CreationDate.str[:4] )
    
    # Pogrupowanie po PostId i Year, i utworzenie zestawienia z licznością każdej grupy; zmiana nazw kolumn
    UpVotesPerYear = UpVotesPerYear.groupby(by=["PostId", "Year"]).size().reset_index()
    UpVotesPerYear.columns = ["PostId", "Year", "Count"]
    
    # Wybranie kolumn i połączenie Posts, które mają PostTypeId równy 1, z UpVotesPerYear
    Posts = Posts.loc[Posts.PostTypeId==1][["Id", "Title"]]
    Posts = Posts.merge(UpVotesPerYear, left_on="Id", right_on="PostId", how="inner")
    
    # Grupowanie po Year i wybranie wiersza z maksymalnym Count - zapisujemy indeksy, które to spełniają
    idx = Posts.groupby("Year")["Count"].transform(max) == Posts["Count"]
    
    # Wybranie wierszy i kolumn, reset indeksów
    Posts = Posts[idx].reset_index()[["Title", "Year", "Count"]]
    
    return Posts

function3(Posts, Votes)

,Title,Year,Count
0,"OK we're all adults here, so really, how on earth should I use a squat toilet?",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up window shades during takeoff and landing?,2013,103
3,How do you know if Americans genuinely/literally mean what they say?,2014,180
4,Immigration officer that stopped me at the airport is texting me. What do I do?,2015,119
5,I don't know my nationality. How can I visit Denmark?,2016,135
6,Why prohibit engine braking?,2017,178


Sprawdźmy czy te ramki danych są identyczne:

In [14]:
function3(Posts, Votes).equals(Cel)

True

## Funkcja 4
Docelowe zapytanie i wynikowa tabela wyglądają następująco:

In [15]:
Cel = pd.read_sql_query("""
                        SELECT
                            Questions.Id,
                            Questions.Title,
                            BestAnswers.MaxScore,
                            Posts.Score AS AcceptedScore,
                            BestAnswers.MaxScore-Posts.Score AS Difference
                        FROM (
                                SELECT Id, ParentId, MAX(Score) AS MaxScore
                                FROM Posts
                                WHERE PostTypeId==2
                                GROUP BY ParentId
                            ) AS BestAnswers
                        JOIN (
                                SELECT * FROM Posts
                                WHERE PostTypeId==1
                            ) AS Questions
                            ON Questions.Id=BestAnswers.ParentId
                        JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
                        WHERE Difference>50
                        ORDER BY Difference DESC
                        """, conn)
Cel

,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if your train station doesn't have any working ticket machines?",126,33,93
1,81376,What is way to eat rice with hands in front of westerners such that it doesn't appear to be yucky?,120,30,90
2,76737,Why do many countries in the world still require citizens of states with a high HDI to get visas?,118,31,87
3,38177,How do you know if Americans genuinely/literally mean what they say?,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in California. How can I settle the case with the court?,101,25,76
5,81492,Why is international first class much more expensive than international economy class?,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,"What's an easy way of making my luggage unique, so that it's easy to spot on the luggage carousel?",72,16,56


To zapytanie pokazuje *ID* i tytuły *pytań*, w których różnica między oceną zaakceptowanej odpowiedzi, a oceną najlepszej odpowiedzi, przekracza 50 (posortowane malejąco według tej różnicy).

Rozwiązanie w `pandas` może wyglądać następująco:

In [16]:
def function4(Users, Posts):
    
    # Utworzenie tabeli BestAnswers, która jest zestawieniem maksymalnego Score dla każdego ParentId
    BestAnswers = Posts[ Posts.PostTypeId==2 ].groupby("ParentId")["Score"].max().reset_index()
    BestAnswers.columns = ["ParentId", "MaxScore"] 
        
    # Utworzenie tabeli Questions, która zawiera posty typu "1" i połączenie jej z BestAnswers
    Questions = Posts[ Posts.PostTypeId==1 ].merge(BestAnswers, left_on="Id", right_on="ParentId", how="inner")
    Questions = Questions[["AcceptedAnswerId", "Id", "Title", "MaxScore"]]
    # Połączenie Questions z Posts
    Posts = Posts.merge(Questions, left_on="Id", right_on="AcceptedAnswerId", how="inner")
    
    # Stworzenie kolumny Difference, wybranie Difference>50 i posortowanie wyników względem Difference
    Posts = Posts.assign(Difference=(Posts.MaxScore-Posts.Score))
    Posts = Posts[ Posts.Difference>50 ].sort_values("Difference", ascending=False)
    
    # Wybranie odpowiednich kolumn, reset indeksów, zmiana nazw kolumn
    Posts = Posts.reset_index()[["Id_y", "Title_y", "MaxScore", "Score", "Difference"]]
    Posts.columns = ["Id", "Title", "MaxScore", "AcceptedScore", "Difference"]
    
    return Posts

function4(Users, Posts)

,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if your train station doesn't have any working ticket machines?",126,33,93
1,81376,What is way to eat rice with hands in front of westerners such that it doesn't appear to be yucky?,120,30,90
2,76737,Why do many countries in the world still require citizens of states with a high HDI to get visas?,118,31,87
3,38177,How do you know if Americans genuinely/literally mean what they say?,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in California. How can I settle the case with the court?,101,25,76
5,81492,Why is international first class much more expensive than international economy class?,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,"What's an easy way of making my luggage unique, so that it's easy to spot on the luggage carousel?",72,16,56


Sprawdźmy czy te ramki danych są identyczne:

In [17]:
function4(Users, Posts).equals(Cel)

True

## Funkcja 5
Docelowe zapytanie i wynikowa tabela wyglądają następująco:

In [18]:
Cel = pd.read_sql_query("""
                        SELECT
                        Posts.Title,
                        CmtTotScr.CommentsTotalScore
                        FROM (
                            SELECT
                            PostID,
                            UserID,
                            SUM(Score) AS CommentsTotalScore
                            FROM Comments
                            GROUP BY PostID, UserID
                        ) AS CmtTotScr
                        JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
                        WHERE Posts.PostTypeId=1
                        ORDER BY CmtTotScr.CommentsTotalScore DESC
                        LIMIT 10
                        """, conn)
Cel

,Title,CommentsTotalScore
0,"How to intentionally get denied entry to the US, without getting into trouble?",75
1,How can I deal with people asking to switch seats with me on a plane?,32
2,What is France's traditional costume?,26
3,What's the longest scheduled public bus ride in the world?,25
4,Can I have a watermelon in hand luggage?,25
5,"How does President Trump's travel ban affect nationals of Iran, Iraq, Libya, Somalia, Sudan, Syria or Yemen coming to the US for tourism/transit/etc?",25
6,Caught speeding 111 Mph (179 km/h) in California. How can I settle the case with the court?,24
7,Returning US Citizen lost passport in Canada,23
8,Legalities and safety concerns of visiting prostitutes in Amsterdam,20
9,India just demonetized all Rs 500 & 1000 notes. What do I do with my leftover cash?,20


To zapytanie pokazuje tytuły postów, których autor zdobył najwiekszą sumę ocen komentarzy pod nim.

Rozwiązanie w `pandas` może wyglądać następująco:

In [19]:
def function5(Posts, Comments):
    
    # Wybranie potrzebnych kolumn z tabeli Comments
    CmtTotScr = Comments[["PostId", "UserId", "Score"]]
    
    # Grupowanie po PostId i UserId i stworzenie sumy Score; zmiana zanw kolumn
    CmtTotScr = CmtTotScr.groupby(["PostId", "UserId"]).agg(np.sum).reset_index()
    CmtTotScr.columns = ["PostId", "UserId", "CommentsTotalScore"]
    
    # Wybranie kolumn i połączenie Posts, które mają PostTypeId równy 1, z UpVotesPerYear
    Posts = Posts.loc[Posts.PostTypeId==1][["Id", "OwnerUserId", "Title"]]
    Posts = Posts.merge(CmtTotScr, left_on=["Id", "OwnerUserId"], right_on=["PostId", "UserId"], how="inner")
    
    # Sortowanie po CommentsTotalScore; wybór kolumn, 10 pierwszych wierszy, oraz reset indeksów
    Posts = Posts.sort_values("CommentsTotalScore", ascending=False)
    Posts = Posts.head(10).reset_index()[["Title", "CommentsTotalScore"]]
    
    return Posts

function5(Posts, Comments)

,Title,CommentsTotalScore
0,"How to intentionally get denied entry to the US, without getting into trouble?",75
1,How can I deal with people asking to switch seats with me on a plane?,32
2,What is France's traditional costume?,26
3,Can I have a watermelon in hand luggage?,25
4,"How does President Trump's travel ban affect nationals of Iran, Iraq, Libya, Somalia, Sudan, Syria or Yemen coming to the US for tourism/transit/etc?",25
5,What's the longest scheduled public bus ride in the world?,25
6,Caught speeding 111 Mph (179 km/h) in California. How can I settle the case with the court?,24
7,Returning US Citizen lost passport in Canada,23
8,India just demonetized all Rs 500 & 1000 notes. What do I do with my leftover cash?,20
9,Legalities and safety concerns of visiting prostitutes in Amsterdam,20


Sprawdźmy czy te ramki danych są identyczne:

In [20]:
function5(Posts, Comments).equals(Cel)

False

Odpowiedź wyszła fałszywa, co widać również gołym okiem przy porównaniu tych ramek - np. w wierszu o indeksie 8. Bierze się to stąd, że w obrębie jednego wyniku *CommentsTotalScore* pojawiają się różne kolejności pytań (identyczna sytuacja ma miejsce w zapytaniu 2). Gdy zignorujemy ten szczegół, te tabele są identyczne.

## Funkcja 6
Docelowe zapytanie i wynikowa tabela wyglądają następująco:

In [21]:
Cel = pd.read_sql_query("""
                        SELECT DISTINCT
                            Users.Id,
                            Users.DisplayName,
                            Users.Reputation,
                            Users.Age,
                            Users.Location
                        FROM (
                                SELECT
                                    Name, UserID
                                FROM Badges
                                WHERE Name IN (
                                    SELECT
                                        Name
                                    FROM Badges
                                    WHERE Class=1
                                    GROUP BY Name
                                    HAVING COUNT(*) BETWEEN 2 AND 10
                                )
                                AND Class=1
                            ) AS ValuableBadges
                        JOIN Users ON ValuableBadges.UserId=Users.Id
                        """, conn)
Cel

,Id,DisplayName,Reputation,Age,Location
0,108,Ankur Banerjee,31273.0,27.0,"London, UK"
1,19,VMAtm,18556.0,33.0,"Tampa, FL, United States"
2,101,Mark Mayo,121667.0,37.0,"Sydney, New South Wales, Australia"
3,466,iHaveacomputer,8360.0,NaN,Down underer
4,793,mindcorrosive,10531.0,32.0,Bulgaria
5,693,RoflcoptrException,33300.0,NaN,None
6,6669,Relaxed,69405.0,NaN,None
7,1737,Gayot Fow,70237.0,NaN,"London, United Kingdom"
8,39065,Pont,1004.0,NaN,Austria
9,19400,phoog,34342.0,50.0,"New York, NY"


To zapytanie pokazuje dane uzytkownikow, ktoórzy zdobyli medale klasy *1*, które zostały zdobyte od 2 do 10 razy.

Rozwiązanie w `pandas` może wyglądać następująco:

In [22]:
def function6(Users, Badges):
    
    # Utworzenie tabeli FCB (FirstClassBadges) pomocniczej do późniejszego VB (ValuableBadges)
    # Wybranie Badges z Class równym 1; utworzenie zestawienia - zliczenie występowania każdej nazwy
    FCB = Badges[ Badges.Class==1 ].groupby("Name").size().reset_index()
    
    # Zmiana nazw kolumn i wybranie Bagdes, które mają Count miedzy 2 a 10
    FCB.columns = ["Name", "Count"]
    FCB = FCB[ (FCB.Count>=2) & (FCB.Count<=10) ]
    
    # Utworzenie tabeli VB - wybór potrzebnych kolumn i filtrowanie
    VB = Badges[["Name", "UserId", "Class"]].loc[ (Badges.Class==1) & (Badges.Name.isin(FCB.Name)) ]
    
    # Połączenie VB z Users
    Users = Users.merge(VB, left_on="Id", right_on="UserId", how="inner")
    
    # Wybranie kolumn i wybranie unikatowych wierszy
    Users = Users.loc[:, ["Id", "DisplayName", "Reputation", "Age", "Location"]].drop_duplicates().reset_index()
    
    # Reset indeksów i wybranie kolumn
    Users = Users.reset_index()[["Id", "DisplayName", "Reputation", "Age", "Location"]]
    
    return Users

function6(Users, Badges)

,Id,DisplayName,Reputation,Age,Location
0,19,VMAtm,18556.0,33.0,"Tampa, FL, United States"
1,101,Mark Mayo,121667.0,37.0,"Sydney, New South Wales, Australia"
2,108,Ankur Banerjee,31273.0,27.0,"London, UK"
3,466,iHaveacomputer,8360.0,NaN,Down underer
4,693,RoflcoptrException,33300.0,NaN,NaN
5,793,mindcorrosive,10531.0,32.0,Bulgaria
6,1737,Gayot Fow,70237.0,NaN,"London, United Kingdom"
7,6669,Relaxed,69405.0,NaN,NaN
8,19400,phoog,34342.0,50.0,"New York, NY"
9,39065,Pont,1004.0,NaN,Austria


Sprawdźmy czy te ramki danych są identyczne:

In [23]:
function6(Users, Badges).equals(Cel)

False

Odpowiedź wyszła fałszywa, co widać również gołym okiem przy porównaniu tych ramek - np. w wierszu o indeksie 0. Bierze się to stąd, że w obrębie tego samego zapytania pojawiają się różne kolejności użytkowników, ponieważ nie zostały w żadnym miejscu ustalone (identyczna sytuacja ma miejsce w zapytaniu 2). Gdy zignorujemy ten szczegół, te tabele są identyczne.

## Funkcja 7
Docelowe zapytanie i wynikowa tabela wyglądają następująco:

In [24]:
Cel = pd.read_sql_query("""
                        SELECT
                            Posts.Title,
                            VotesByAge2.OldVotes
                        FROM Posts
                        JOIN (
                            SELECT
                                PostId,
                                MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
                                MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
                                SUM(Total) AS Votes
                            FROM (
                                SELECT
                                    PostId,
                                    CASE STRFTIME('%Y', CreationDate)
                                        WHEN '2017' THEN 'new'
                                        WHEN '2016' THEN 'new'
                                        ELSE 'old'
                                        END VoteDate,
                                    COUNT(*) AS Total
                                FROM Votes
                                WHERE VoteTypeId=2
                                GROUP BY PostId, VoteDate
                            ) AS VotesByAge
                            GROUP BY VotesByAge.PostId
                            HAVING NewVotes=0
                        ) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
                        WHERE Posts.PostTypeId=1
                        ORDER BY VotesByAge2.OldVotes DESC
                        LIMIT 10
                        """, conn)
Cel

,Title,OldVotes
0,Which European cities have bike rental stations for tourists?,39
1,Why do hostels require you to 'rent' bedding?,28
2,What to do with your valuables on a low-cost holiday while swimming/diving in Central America?,25
3,Can't check-in to a hotel because I am 18,25
4,What are some good ways to find things to explore on-site in an unfamiliar place?,24
5,Alarm Clock without Noise? To wake up in common sleeping rooms and airports without noise?,24
6,What times of the year are best for visiting France?,23
7,What is the business model of commercial free walking tours?,23
8,Getting work on a cruise ship in order to travel,23
9,Carrying medicines internationally for a friend,23


To zapytanie pokazuje tytuły postów typu 1, które mają najwięcej "*starych*" głosów typu 2, i nie mają "*nowych*" głosów.

Rozwiązanie w `pandas` może wyglądać następująco:

In [25]:
def age(x):
    # Funkcja pomocnicza do function7
    # zwraca "new", jeśli x=="2017" lub x=="2018"
    # w przeciwnym wypadku zwraca "old"
    y = np.repeat("none", len(x))
    y[ (x=="2017") | (x=="2016") ] = "new"
    y[ (x!="2017") & (x!="2016") ] = "old"
    
    return y
    
def function7(Posts, Votes):
    
    # Utworzenie tabeli VotesByAge - wybranie wierszy z VoteType==2, przydzielenie "wieku" i utworzenie zestawienia
    VotesByAge = Votes[ Votes.VoteTypeId==2 ].assign( VoteDate=Votes.CreationDate.str[:4] )
    VotesByAge = VotesByAge.assign( VoteDate=age(VotesByAge.VoteDate) )
    VotesByAge = VotesByAge.groupby(["PostId", "VoteDate"]).size().reset_index()
    VotesByAge.columns = ["PostId", "VoteDate", "Total"]
    
    # Utworzenie tabeli z "nowymi" głosami
    NewVotes = VotesByAge[ VotesByAge.VoteDate=="new" ].groupby("PostId")["Total"].max().reset_index()
    NewVotes.columns = ["PostId", "NewVotes"]
    
    # Utworzenie tabeli z "starymi" głosami
    OldVotes = VotesByAge[ VotesByAge.VoteDate=="old" ].groupby("PostId")["Total"].max().reset_index()
    OldVotes.columns = ["PostId", "OldVotes"]
    
    # Wybranie postów typu "1"
    Posts = Posts[ Posts.PostTypeId==1 ]
    
    # Połączenie tabel OldVotes i NewVotes
    OldVotes = OldVotes.merge(NewVotes, left_on="PostId", right_on="PostId", how="outer")
    
    # Wybranie rekordów, gdzie nie ma "nowych głosów"
    OldVotes = OldVotes[ np.isnan(OldVotes.NewVotes) ]
    
    # Połączenie tabeli z tabelą Posts, wybranie odpowiednich kolumn i wierszy, sortowanie i reset indeksów
    OldVotes = OldVotes.merge(Posts, left_on="PostId", right_on="Id", how="inner")
    OldVotes = OldVotes.sort_values("OldVotes", ascending=False)
    OldVotes = OldVotes.head(10).reset_index()[["Title", "OldVotes"]]
    
    return OldVotes

function7(Posts, Votes)

,Title,OldVotes
0,Which European cities have bike rental stations for tourists?,39.0
1,Why do hostels require you to 'rent' bedding?,28.0
2,What to do with your valuables on a low-cost holiday while swimming/diving in Central America?,25.0
3,Can't check-in to a hotel because I am 18,25.0
4,What are some good ways to find things to explore on-site in an unfamiliar place?,24.0
5,Alarm Clock without Noise? To wake up in common sleeping rooms and airports without noise?,24.0
6,Should I avoid overnight flights from Dubai to India?,23.0
7,"Can a visitor who is ""of age"" in their country drink underage in the United States of America?",23.0
8,What is the business model of commercial free walking tours?,23.0
9,Carrying medicines internationally for a friend,23.0


Sprawdźmy czy te ramki danych są identyczne:

In [26]:
function7(Posts, Votes).equals(Cel)

False

Odpowiedź wyszła fałszywa, co widać również gołym okiem przy porównaniu tych ramek - np. w wierszu o indeksie 6. Bierze się to stąd, że w obrębie tej samej wartości *OldVotes* pojawiają się różne kolejności tytułów, ponieważ nie zostały w żadnym miejscu ustalone (identyczna sytuacja ma miejsce w zapytaniu 2). Gdy zignorujemy ten szczegół, te tabele są identyczne.